In [1]:
import bisect
import sys
from test_functions import test_functions
import numpy as np
import matplotlib.pyplot as plt
rec = 100000
sys.setrecursionlimit(rec)

In [2]:
def F(func, dots, L, x):
    f_s = [func(i) - L * np.abs(x - i) for (i, j) in dots]
    return max(f_s)

In [3]:
def na(func, dots, L, counter):
    min_value, min_index = min([(y, i) for i, (x, y) in enumerate(dots)])

    if abs(dots[min_index][0] - dots[min_index + 1][0]) < epsilon:
        func_values = [func(x) for (x, y) in dots]
        return min(func_values), counter
    counter += 1
    new_dot = (dots[min_index][0] + dots[min_index + 1][0]) / 2 - (func(dots[min_index][0]) - \
                                                                   func(dots[min_index + 1][0])) / (2 * L)
    new_index = bisect.bisect_left(dots, (new_dot, 0))
    bisect.insort_left(dots, (new_dot, 0))
    new_char_1 = (func(dots[new_index][0]) + func(dots[new_index - 1][0])) / 2 - \
               L * (dots[new_index][0] - dots[new_index - 1][0])
    new_char_2 = (func(dots[new_index][0]) + func(dots[new_index + 1][0])) / 2 - \
                 L * (dots[new_index + 1][0] - dots[new_index][0])
    dots[new_index - 1] = (dots[new_index - 1][0], new_char_1)
    dots[new_index] = (new_dot, new_char_2)
    return na(func, dots, L, counter)



In [4]:
def ng(func, dots, counter, H_max_index, tau = 2):
    min_value, min_index = min([(y, i) for i, (x, y, h) in enumerate(dots)])
    if abs(dots[min_index][0] - dots[min_index + 1][0]) < epsilon:
        func_values = [func(x) for (x, y, h) in dots]
        return min(func_values), counter, dots[H_max_index][2]
    L = 1 if dots[H_max_index][2] == 0 else tau * dots[H_max_index][2]
    counter += 1
    new_dot = (dots[min_index][0] + dots[min_index + 1][0]) / 2 - (func(dots[min_index][0]) -
                                                                   func(dots[min_index + 1][0])) / (2 * L)
    new_index = bisect.bisect_left(dots, (new_dot, 0, 0))
    bisect.insort_left(dots, (new_dot, 0, 0))
    H_1 = abs(func(dots[new_index][0]) - func(dots[new_index - 1][0])) / (dots[new_index][0] - dots[new_index - 1][0])
    H_2 = abs(func(dots[new_index + 1][0]) - func(dots[new_index][0])) / (dots[new_index + 1][0] - dots[new_index][0])
    dots[new_index - 1] = (dots[new_index - 1][0], 0, H_1)
    dots[new_index] = (new_dot, 0, H_2)
    if new_index - H_max_index == 1:
        max_value, H_max_index = max([(h, -i) for i, (x, y, h) in enumerate(dots)])
        H_max_index = H_max_index * (-1)
        L = 1 if dots[H_max_index][2] == 0 else tau * dots[H_max_index][2]
    new_char_1 = (func(dots[new_index][0]) + func(dots[new_index - 1][0])) / 2 - \
                 L * (dots[new_index][0] - dots[new_index - 1][0])
    new_char_2 = (func(dots[new_index][0]) + func(dots[new_index + 1][0])) / 2 - \
                 L * (dots[new_index + 1][0] - dots[new_index][0])
    dots[new_index - 1] = (dots[new_index - 1][0], new_char_1, H_1)
    dots[new_index] = (new_dot, new_char_2, H_2)
    return ng(func, dots, counter, H_max_index, tau)


In [5]:
def i_ng_1(func, dots, counter, H, tau = 2):
    max_value, max_index = max([(y, -i) for i, (x, y) in enumerate(dots)])
    if abs(dots[max_index][0] - dots[max_index + 1][0]) < epsilon:
        func_values = [func(x) for (x, y) in dots]
        return min(func_values), counter, H
    L = 1 if H == 0 else tau * H
    counter += 1
    new_dot = (dots[max_index][0] + dots[max_index + 1][0]) / 2 - \
              (func(dots[max_index + 1][0]) - func(dots[max_index][0])) / (2 * L)
    new_index = bisect.bisect_left(dots, (new_dot, 0))
    bisect.insort_left(dots, (new_dot, 0))
    new_char_1 = L * (dots[max_index][0] - dots[max_index - 1][0]) + ((func(dots[max_index][0]) - func(dots[max_index - 1][0])) ** 2) / (L * (dots[max_index][0] - dots[max_index - 1][0])) - 2 * (func(dots[max_index][0]) - func(dots[max_index - 1][0]))
    new_char_2 = L * (dots[max_index + 1][0] - dots[max_index][0]) + ((func(dots[max_index + 1][0]) - func(dots[max_index][0])) ** 2) / (L * (dots[max_index + 1][0] - dots[max_index][0])) - 2 * (func(dots[max_index + 1][0]) - func(dots[max_index][0]))
    dots[new_index - 1] = (dots[new_index - 1][0], new_char_1)
    dots[new_index] = (new_dot, new_char_2)
    H_1 = abs(func(dots[new_index][0]) - func(dots[new_index - 1][0])) / (dots[new_index][0] - dots[new_index - 1][0])
    H_2 = abs(func(dots[new_index + 1][0]) - func(dots[new_index][0])) / (dots[new_index + 1][0] - dots[new_index][0])
    if max(H_1, H_2) > H:
        H = max(H_1, H_2)
    return i_ng_1(func, dots, counter, H, tau)


In [10]:
for index, func in enumerate(test_functions):
    a, b, L, counter = func.interval[0], func.interval[1], func.L, 0
    epsilon = (b - a) * (10 ** -4)
    char_a = (func.f(a) + func.f(b)) / 2 - L * (b - a) / 2
    dots = [(a, char_a), (b, float("inf"))]
    minval, count = na(func.f, dots, L, counter)
    print(index + 1, minval, count)

1 -29763.23207421922 227
2 -1.899599348242236 190
3 -12.031242882934125 446
4 -3.8504507084903694 680
5 -1.4890720249095521 212
6 -0.8242393171413713 197
7 -1.6013075132043646 216
8 -14.508007872538233 369
9 -1.9059607612889131 191
10 -7.916727183123053 282
11 -1.499999990989576 554
12 -0.9999999993865137 497
13 -1.5874010519372086 1186
14 -0.7886853838324217 183
15 -0.03553390076491803 985
16 7.515924153455886 1156
17 7.0000343945575025 883
18 9.545533802859057e-09 465
19 -7.815674542778331 216
20 -0.06349051994752021 127


In [11]:
xarrays, yarrays, ogarrays = [], [], []
for index, func in enumerate(test_functions):
    x_arr = np.linspace(func.interval[0], func.interval[1], 1200)
    y = []
    og = []
    a, b, L, counter = func.interval[0], func.interval[1], func.L, 0
    epsilon = (b - a) * (10 ** -4)
    char_a = (func.f(a) + func.f(b)) / 2 - L * (b - a) / 2
    dots = [(a, char_a), (b, float("inf"))]
    minval, count = na(func.f, dots, L, counter)
    for i in x_arr:
        y.append(func.f(i))
        og.append(F(func.f, dots, L, i))
    xarrays.append(x_arr)
    yarrays.append(y)
    ogarrays.append(og)

KeyboardInterrupt: 

In [ ]:
 for index in range(20):
    plt.plot(xarrays[index], yarrays[index])
    plt.plot(xarrays[index], ogarrays[index])
    plt.show()

In [18]:
for index, func in enumerate(test_functions):
    tau = 4
    a, b, counter = func.interval[0], func.interval[1], 0
    H = abs(func.f(b) - func.f(a)) / (b - a)
    L = 1 if H == 0 else tau * H
    epsilon = (b - a) * (10 ** -4)
    char_a = (func.f(a) + func.f(b)) / 2 - L * (b - a) / 2
    dots = [(a, char_a, H), (b, float("inf"), -float("inf"))]
    minval, count, H = ng(func.f, dots, counter, 0, tau)
    print(index + 1, minval, count)

1 -29763.232332995936 254
2 -1.8995992895448253 482


IndexError: list index out of range

In [ ]:
xarrays, yarrays, ogarrays = [], [], []
for index, func in enumerate(test_functions):
    x_arr = np.linspace(func.interval[0], func.interval[1], 1200)
    y, og = [], []
    tau = 2
    a, b, counter = func.interval[0], func.interval[1], 0
    H = abs(func.f(b) - func.f(a)) / (b - a)
    L = 1 if H == 0 else tau * H
    epsilon = (b - a) * (10 ** -4)
    char_a = (func.f(a) + func.f(b)) / 2 - L * (b - a) / 2
    dots = [(a, char_a), (b, float("inf"))]
    minval, count, H = ng(func.f, dots, counter, H, tau)
    for i in x_arr:
        y.append(func.f(i))
        og.append(F(func.f, dots, H, i))
    xarrays.append(x_arr)
    yarrays.append(y)
    ogarrays.append(og)

In [ ]:
 for index in range(20):
    plt.plot(xarrays[index], yarrays[index])
    plt.plot(xarrays[index], ogarrays[index])
    plt.show()

In [ ]:
for index, func in enumerate(test_functions):
    tau = 1
    a, b, counter = func.interval[0], func.interval[1], 0
    H = abs(func.f(b) - func.f(a)) / (b - a)
    L = 1 if H == 0 else tau * H
    epsilon = (b - a) * (10 ** -4)
    char_a = L * (b - a) + ((func.f(b) - func.f(a)) ** 2) / (L * (b - a)) - 2 * (func.f(b) - func.f(a))
    dots = [(a, char_a), (b, -float("inf"))]
    maxval, count, H = i_ng_1(func.f, dots, counter, H, tau)
    print(index + 1, maxval, count)

In [ ]:
xarrays, yarrays, ogarrays = [], [], []
for index, func in enumerate(test_functions):
    x_arr = np.linspace(func.interval[0], func.interval[1], 1200)
    y, og = [], []
    tau = 1.5
    a, b, counter = func.interval[0], func.interval[1], 0
    H = abs(func.f(b) - func.f(a)) / (b - a)
    L = 1 if H == 0 else tau * H
    epsilon = (b - a) * (10 ** -4)
    char_a = L * (b - a) + ((func.f(b) - func.f(a)) ** 2) / (L * (b - a)) - 2 * (func.f(b) - func.f(a))
    dots = [(a, char_a), (b, -float("inf"))]
    maxval, count, H = i_ng_1(func.f, dots, counter, H, tau)
    for i in x_arr:
        y.append(func.f(i))
        og.append(F(func.f, dots, H, i))
    xarrays.append(x_arr)
    yarrays.append(y)
    ogarrays.append(og)

In [ ]:
 for index in range(20):
    plt.plot(xarrays[index], yarrays[index])
    plt.plot(xarrays[index], ogarrays[index])
    plt.show()